In [63]:
def ln():
    print("-----------------------------------------------------------------------------------------------------------------")
    

In [379]:
class ECCSys:
    def __init__(self,a,b,c,p):
        ln()
        print("[INIT]: Initializing ECC System")
        print("Equation: ", f'y^2 = {a}x^3 + {b}x + {c}')
        print("Modulus: ", p)
        ln()
        self.W = None
        
        self.a = a
        self.b = b
        self.c = c
        self.p = p 
        self.points = []
        self.squareRts = dict((i,[]) for i in range(p))
        self.getSquares()
        self.getPoints()
        self.WOperations = dict((i,None) for i in range(len(self.points)+1))
        self.WPointsToMultiples = {}
    
    
    def evaluateX(self,x, verbose = False):
        p = self.p
        ySq = (self.a*(x**3)+self.b*x+self.c)%self.p
        pts = [(x,rt) for rt in self.squareRts[ySq]]
        if(verbose):
            print("For x =",x,"  ------>    y^2 =",ySq, end = "\n")
            for i in pts:
                print("    pt:",i)
        return pts
        
    def getPoints(self):
        print("[PROCESS]: Get all points")
        for i in range(self.p):
            pts = self.evaluateX(i, True)
            if(pts!=[]):
                for pt in pts:
                    self.points.append(pt)
        ln()
        print("[RESULTS]: Showing all points")
        print(self.points)
        ln()
            
    def getSquares(self):
        print("[PROCESS]: Getting square roots of all numbers mod", self.p)
        for i in range(self.p):
            
            squared = (i**2)%self.p
            print(i,"^2=", squared, "mod",self.p,"       So sqrt(", squared, ") is ", i)
            self.squareRts[squared].append(i)
        ln()
        print("[RESULTS]: Showing dictionary of numbers and their square roots mod", self.p)
        print(self.squareRts)
        ln()
        
    def inverse(self, pt, verbose = False):
        try:
            assert(type(pt)==tuple and pt in self.points)
        except ValueError:
            return "W must be of type tuple and a valid point"
        if(verbose):
            print("[PROCESS]: Find inverse of point", pt)
        x = pt[0]
        for p in self.points:
            if(p!=pt and p[0]==x):
                if(verbose):
                    print("Inverse is",p)
                return p
        if(verbose):
            print("Inverse is",pt)
        return pt
    
    
    def setW(self, W):
        try:
            assert(type(W)==tuple and W in self.points)
            print("[PROCESS]: Setting W to", W, "and calculating W multiples")
            self.W = W
            self.WAddition()
            return
        except ValueError:
            print( W in self.points)
            print(type(W)==tuple)
            return "W must be of type tuple and a valid point"
        
        
    def WAddition(self):
        self.WOperations[0] = 0
        self.WPointsToMultiples[0] = 0
        print("0W = 0")
        W = self.W
        print("1W =", W)
        last = W
        self.WOperations[1] = last
        self.WPointsToMultiples[last] = 1
        for i in range(len(self.points)-1):
            iteration = i+2
            last = self.addPoints(W,last)
            print(str(iteration)+"W =", last)
            self.WOperations[iteration] = last
            self.WPointsToMultiples[last] = iteration
        ln()
            
    def addPoints(self,p1,p2):
        try:
            assert(type(p1)==tuple and type(p2)==tuple)
        except:
            return "Points must be of type tuple"
        
        if(p1!=p2):
            m = ((p2[1]-p1[1])%self.p)*modinv((p2[0]-p1[0])%self.p, self.p)%self.p
            x = (m**2-p1[0]- p2[0])%self.p
            y = (m*(p1[0]-x)-p1[1])%self.p
            
            return(x,y)
        else:
            
            x = (((3*p1[0]**2 + self.b) * modinv(2*p1[1], self.p))**2 - 2*p1[0])%self.p
            y = ((3*p1[0]**2+self.b)*modinv(2*p1[1],self.p)*(p1[0]-x)-p1[1])%self.p
            
            return (x,y)

        
    def WMultipleToPoint(self,c, verbose=False):
        if(verbose):
            print("[PROCESS]: Find " +str(c)+"W")
        finalC = (c)%(len(self.WOperations))
        WOp = self.WOperations[finalC]
        if(verbose):
            print(str(c)+"W =", str(finalC)+"W =", WOp)
            ln()
        return WOp
    
    
    def WPointToMultiple(self, pt):
        return self.WPointsToMultiples[pt]



def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [380]:
sys = ECCSys(1,2,4,11)

-----------------------------------------------------------------------------------------------------------------
[INIT]: Initializing ECC System
Equation:  y^2 = 1x^3 + 2x + 4
Modulus:  11
-----------------------------------------------------------------------------------------------------------------
[PROCESS]: Getting square roots of all numbers mod 11
0 ^2= 0 mod 11        So sqrt( 0 ) is  0
1 ^2= 1 mod 11        So sqrt( 1 ) is  1
2 ^2= 4 mod 11        So sqrt( 4 ) is  2
3 ^2= 9 mod 11        So sqrt( 9 ) is  3
4 ^2= 5 mod 11        So sqrt( 5 ) is  4
5 ^2= 3 mod 11        So sqrt( 3 ) is  5
6 ^2= 3 mod 11        So sqrt( 3 ) is  6
7 ^2= 5 mod 11        So sqrt( 5 ) is  7
8 ^2= 9 mod 11        So sqrt( 9 ) is  8
9 ^2= 4 mod 11        So sqrt( 4 ) is  9
10 ^2= 1 mod 11        So sqrt( 1 ) is  10
-----------------------------------------------------------------------------------------------------------------
[RESULTS]: Showing dictionary of numbers and their square roots mod 11
{0: 

In [381]:
sys.setW((0,2))

[PROCESS]: Setting W to (0, 2) and calculating W multiples
0W = 0
1W = (0, 2)
2W = (3, 2)
3W = (8, 9)
4W = (6, 1)
5W = (9, 5)
6W = (7, 3)
7W = (2, 4)
8W = (10, 10)
9W = (10, 1)
10W = (2, 7)
11W = (7, 8)
12W = (9, 6)
13W = (6, 10)
14W = (8, 2)
15W = (3, 9)
16W = (0, 9)
-----------------------------------------------------------------------------------------------------------------


In [382]:
sys.WMultipleToPoint(20,True)

[PROCESS]: Find 20W
20W = 3W = (8, 9)
-----------------------------------------------------------------------------------------------------------------


(8, 9)

In [383]:
sys.WPointToMultiple((0,2))

1

In [384]:
sys.inverse((3,9), True)

[PROCESS]: Find inverse of point (3, 9)
Inverse is (3, 2)


(3, 2)

## ECCSys Functions Explained

#### 1. Intialization: sys = ECCSys(a,b,c,p)

Creates ECC System with parameters that correspond to $y^2 = ax^3 + bx + c$ with modulus $p$.

#### 2. sys.setW(w)

Set point $W$ to $w$ in system. $W$ is starting point of ECC. Automatically calculates all multiples of point W less than p to make it easy to add points together. 

#### 3. sys.WMultipleToPoint(c)

Calculates $c*W$. Returns  $pt\in (\mathbf{Z}_p,\mathbf{Z}_p)$ in form of tuple.

#### 4. sys.WPointToMultiple(pt)

Calculates the multiple $c$ such that $c*W = pt$ where $pt\in (\mathbf{Z}_p,\mathbf{Z}_p)$. Used to avoid making a function that multiplies points together. That would probably be a lot of work. Returns integer.

#### 5. sys.inverse(pt)

Calculates inverse of pt. Returns $pt\in (\mathbf{Z}_p,\mathbf{Z}_p)$ in form of tuple.

#### 6.sys.addPoints(pt1,pt2)

Adds to points together in system. Returns $pt\in (\mathbf{Z}_p,\mathbf{Z}_p)$ in form of tuple.



In [397]:
#Test with alicia/bunbury example from notes

W = (0,2)
sys.setW(W)
hBunbury = 4
hW = sys.WMultipleToPoint(4, True)




[PROCESS]: Setting W to (0, 2) and calculating W multiples
0W = 0
1W = (0, 2)
2W = (3, 2)
3W = (8, 9)
4W = (6, 1)
5W = (9, 5)
6W = (7, 3)
7W = (2, 4)
8W = (10, 10)
9W = (10, 1)
10W = (2, 7)
11W = (7, 8)
12W = (9, 6)
13W = (6, 10)
14W = (8, 2)
15W = (3, 9)
16W = (0, 9)
-----------------------------------------------------------------------------------------------------------------
[PROCESS]: Find 4W
4W = 4W = (6, 1)
-----------------------------------------------------------------------------------------------------------------


In [392]:
mAlicia = (10,1)
kAlicia = 6
kW = sys.WMultipleToPoint(kAlicia,True)
#Find khW in terms of multiple of W. Then convert multiple to point
khW = sys.WMultipleToPoint(kAlicia*sys.WPointToMultiple(hW),True)   


mPluskhW = sys.addPoints(mAlicia,khW)
print(mPluskhW)

[PROCESS]: Find 6W
6W = 6W = (7, 3)
-----------------------------------------------------------------------------------------------------------------
[PROCESS]: Find 24W
24W = 7W = (2, 4)
-----------------------------------------------------------------------------------------------------------------
(0, 9)


In [395]:
hkW = sys.WMultipleToPoint(hBunbury*sys.WPointToMultiple(kW),True)
originalM = sys.addPoints(mPluskhW,  sys.inverse(hkW))
print(originalM)

[PROCESS]: Find 24W
24W = 7W = (2, 4)
-----------------------------------------------------------------------------------------------------------------
(10, 1)


In [398]:
#Setting up my curve
sys = ECCSys(1,2,9,17)

-----------------------------------------------------------------------------------------------------------------
[INIT]: Initializing ECC System
Equation:  y^2 = 1x^3 + 2x + 9
Modulus:  17
-----------------------------------------------------------------------------------------------------------------
[PROCESS]: Getting square roots of all numbers mod 17
0 ^2= 0 mod 17        So sqrt( 0 ) is  0
1 ^2= 1 mod 17        So sqrt( 1 ) is  1
2 ^2= 4 mod 17        So sqrt( 4 ) is  2
3 ^2= 9 mod 17        So sqrt( 9 ) is  3
4 ^2= 16 mod 17        So sqrt( 16 ) is  4
5 ^2= 8 mod 17        So sqrt( 8 ) is  5
6 ^2= 2 mod 17        So sqrt( 2 ) is  6
7 ^2= 15 mod 17        So sqrt( 15 ) is  7
8 ^2= 13 mod 17        So sqrt( 13 ) is  8
9 ^2= 13 mod 17        So sqrt( 13 ) is  9
10 ^2= 15 mod 17        So sqrt( 15 ) is  10
11 ^2= 2 mod 17        So sqrt( 2 ) is  11
12 ^2= 8 mod 17        So sqrt( 8 ) is  12
13 ^2= 16 mod 17        So sqrt( 16 ) is  13
14 ^2= 9 mod 17        So sqrt( 9 ) is  14
15 ^2

## My Example
Alicia wants to send secret message to Bunbury. They agree to use the elliptic curve created above. They decide W = (0,3) because it is a generator.

In [405]:
W = (0,3)
sys.setW(W)

[PROCESS]: Setting W to (0, 3) and calculating W multiples
0W = 0
1W = (0, 3)
2W = (2, 2)
3W = (11, 11)
4W = (4, 8)
5W = (5, 12)
6W = (3, 12)
7W = (6, 13)
8W = (10, 3)
9W = (7, 14)
10W = (9, 12)
11W = (9, 5)
12W = (7, 3)
13W = (10, 14)
14W = (6, 4)
15W = (3, 5)
16W = (5, 5)
17W = (4, 9)
18W = (11, 6)
19W = (2, 15)
20W = (0, 14)
-----------------------------------------------------------------------------------------------------------------


In [415]:
#Bunbury's public key
bunburyPrivateKey = 10
bunburyPublicKey = sys.WMultipleToPoint(bunburyPrivateKey)
print("Bunbury's public key:",bunburyPublicKey)

Bunbury's public key: (9, 12)


In [416]:
#Encryption process by Alicia
message = [(0,3), (2,2), (4,8), (6,13)]
aliciaPrivateKey = 17
kW = sys.WMultipleToPoint(aliciaPrivateKey)
khW = sys.WMultipleToPoint(aliciaPrivateKey*sys.WPointToMultiple(bunburyPublicKey))
aliciaPublicKey = {'kW':kW, 'MPluskhW': []}
for pt in message:
    aliciaPublicKey['MPluskhW'].append(sys.addPoints(pt,khW))
print("Alicia's public key:",aliciaPublicKey)

Alicia's public key: {'kW': (4, 9), 'MPluskhW': [(11, 11), (4, 8), (3, 12), (7, 14)]}


In [414]:
#Decryption process by Bunbury
hkW = sys.WMultipleToPoint(bunburyPrivateKey*sys.WPointToMultiple(aliciaPublicKey['kW']))
hkWInverse = sys.inverse(hkW)
decryptedMessage = []
for pt in aliciaPublicKey['MPluskhW']:
    decryptedMessage.append(sys.addPoints(pt,hkWInverse))
print("Decrypted message:",decryptedMessage)

Decrypted message: [(0, 3), (2, 2), (4, 8), (6, 13)]
